In [ ]:
import pandas as pd
import time
import math
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load data dari Google Drive
orders = pd.read_csv('/content/drive/MyDrive/olist_orders_dataset.csv')

# Ambil semua kolom, ubah ke string dan hilangkan nilai NaN
columns = orders.columns.tolist()
data_dict = {}

for col in columns:
    col_data = orders[col].dropna().astype(str).tolist()
    data_dict[col] = col_data

# ----------------- SEARCHING ALGORITHMS -----------------
def jump_search(arr, target):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while prev < n and arr[min(step, n)-1] < target:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    for i in range(prev, min(step, n)):
        if arr[i] == target:
            return i
    return -1

def binary_search(arr, target):
    low, high = 0, len(arr)-1
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return -1

def hash_search(data, target):
    index = {v: i for i, v in enumerate(data)}
    return index.get(target, -1)

# ----------------- SORTING ALGORITHMS -----------------
def heapify(arr, n, i):
    largest = i
    l = 2*i + 1
    r = 2*i + 2
    if l < n and arr[l] > arr[largest]:
        largest = l
    if r < n and arr[r] > arr[largest]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

def heap_sort(arr):
    arr = arr.copy()
    n = len(arr)
    for i in range(n//2 - 1, -1, -1):
        heapify(arr, n, i)
    for i in range(n-1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]
        heapify(arr, i, 0)
    return arr

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr)//2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    while left and right:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left + right)
    return result

def selection_sort(arr):
    arr = arr.copy()
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

def bubble_sort(arr):
    arr = arr.copy()
    n = len(arr)
    for i in range(n):
        for j in range(n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

def counting_sort(arr):
    try:
        arr_int = list(map(int, arr))  # Coba ubah ke integer
    except ValueError:
        return arr  # Jika gagal, kembalikan data asli (tidak cocok)

    max_val = max(arr_int)
    count = [0] * (max_val + 1)
    for num in arr_int:
        count[num] += 1
    sorted_arr = []
    for i, c in enumerate(count):
        sorted_arr.extend([i] * c)
    return list(map(str, sorted_arr))  # Kembalikan ke string

# ----------------- KOMBINASI SEARCH + SORT -----------------
def test_all_combinations():
    print("\n🔀 UJI KOMBINASI SORTING + SEARCHING\n")

    combinations = [
        ("Jump Search", jump_search, True, "Heap Sort", heap_sort),
        ("Jump Search", jump_search, True, "Merge Sort", merge_sort),
        ("Hash Search", hash_search, False, "Selection Sort", selection_sort),
        ("Jump Search", jump_search, True, "Bubble Sort", bubble_sort),
        ("Binary Search", binary_search, True, "Counting Sort", counting_sort),
    ]

    total_times = {}  # Untuk menyimpan total waktu dari setiap kombinasi

    for col, values in data_dict.items():
        if not values:
            continue
        print(f"📂 Kolom: {col}")
        target = values[len(values) // 2]  # Nilai tengah sebagai target

        for s_name, s_func, s_needs_sort, sort_name, sort_func in combinations:
            combo_key = f"{s_name} + {sort_name}"
            test_data = values.copy()

            # Counting Sort: periksa apakah bisa dilakukan
            if sort_name == "Counting Sort":
                try:
                    _ = list(map(int, test_data))
                    can_counting_sort = True
                except ValueError:
                    can_counting_sort = False
                if not can_counting_sort:
                    print(f"  ⚠️ {sort_name} tidak bisa untuk kolom ini (data bukan integer)")
                    # Tetap lakukan searching
                    search_start = time.time()
                    idx = s_func(test_data, target)
                    search_time = time.time() - search_start
                    total = search_time
                    total_times[combo_key] = total_times.get(combo_key, 0) + total
                    print(f"  🔁 {combo_key} → Index: {idx}, Sort: skipped, Search: {search_time:.6f}s")
                    continue

            # Sorting
            sort_start = time.time()
            sorted_data = sort_func(test_data)
            sort_time = time.time() - sort_start

            # Validasi hasil sorting jika diperlukan
            #if s_needs_sort and sorted_data != sorted(test_data):
             #   print(f"  ⚠️ Kombinasi {sort_name} + {s_name} gagal (data tidak terurut benar)")
              #  continue

            # Searching
            search_start = time.time()
            idx = s_func(sorted_data if s_needs_sort else test_data, target)
            search_time = time.time() - search_start

            # Simpan total waktu
            total = sort_time + search_time
            total_times[combo_key] = total_times.get(combo_key, 0) + total

            print(f"  🔁 {combo_key} → Index: {idx}, Sort: {sort_time:.6f}s, Search: {search_time:.6f}s")

        print()

    # Cetak total waktu kombinasi
    print("\n📊 TOTAL WAKTU SETIAP KOMBINASI:")
    for combo, total in total_times.items():
        print(f"  ⏱️ {combo}: {total:.6f} detik")


# ----------------- JALANKAN UJI -----------------
test_all_combinations()


Mounted at /content/drive

🔀 UJI KOMBINASI SORTING + SEARCHING

📂 Kolom: order_id
  🔁 Jump Search + Heap Sort → Index: 62700, Sort: 0.674049s, Search: 0.000178s
  🔁 Jump Search + Merge Sort → Index: 62700, Sort: 1.184783s, Search: 0.000154s
  🔁 Hash Search + Selection Sort → Index: 49720, Sort: 612.698904s, Search: 0.033521s
  🔁 Jump Search + Bubble Sort → Index: 62700, Sort: 1028.415755s, Search: 0.000283s
  ⚠️ Counting Sort tidak bisa untuk kolom ini (data bukan integer)
  🔁 Binary Search + Counting Sort → Index: 49720, Sort: skipped, Search: 0.000034s

📂 Kolom: customer_id
  🔁 Jump Search + Heap Sort → Index: 97363, Sort: 1.207042s, Search: 0.000333s
  🔁 Jump Search + Merge Sort → Index: 97363, Sort: 1.602228s, Search: 0.000215s
  🔁 Hash Search + Selection Sort → Index: 49720, Sort: 615.888853s, Search: 0.032462s
  🔁 Jump Search + Bubble Sort → Index: 97363, Sort: 1043.686723s, Search: 0.000275s
  ⚠️ Counting Sort tidak bisa untuk kolom ini (data bukan integer)
  🔁 Binary Search + C

In [ ]:
import pandas as pd
import time
import math
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load data dari Google Drive
orders = pd.read_csv('/content/drive/MyDrive/olist_products_dataset.csv')

# Ambil semua kolom, ubah ke string dan hilangkan nilai NaN
columns = orders.columns.tolist()
data_dict = {}

for col in columns:
    col_data = orders[col].dropna().astype(str).tolist()
    data_dict[col] = col_data

# ----------------- SEARCHING ALGORITHMS -----------------
def jump_search(arr, target):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while prev < n and arr[min(step, n)-1] < target:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    for i in range(prev, min(step, n)):
        if arr[i] == target:
            return i
    return -1

def binary_search(arr, target):
    low, high = 0, len(arr)-1
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return -1

def hash_search(data, target):
    index = {v: i for i, v in enumerate(data)}
    return index.get(target, -1)

# ----------------- SORTING ALGORITHMS -----------------
def heapify(arr, n, i):
    largest = i
    l = 2*i + 1
    r = 2*i + 2
    if l < n and arr[l] > arr[largest]:
        largest = l
    if r < n and arr[r] > arr[largest]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

def heap_sort(arr):
    arr = arr.copy()
    n = len(arr)
    for i in range(n//2 - 1, -1, -1):
        heapify(arr, n, i)
    for i in range(n-1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]
        heapify(arr, i, 0)
    return arr

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr)//2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    while left and right:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left + right)
    return result

def selection_sort(arr):
    arr = arr.copy()
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

def bubble_sort(arr):
    arr = arr.copy()
    n = len(arr)
    for i in range(n):
        for j in range(n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

def counting_sort(arr):
    try:
        arr_int = list(map(int, arr))  # Coba ubah ke integer
    except ValueError:
        return arr  # Jika gagal, kembalikan data asli (tidak cocok)

    max_val = max(arr_int)
    count = [0] * (max_val + 1)
    for num in arr_int:
        count[num] += 1
    sorted_arr = []
    for i, c in enumerate(count):
        sorted_arr.extend([i] * c)
    return list(map(str, sorted_arr))  # Kembalikan ke string

# ----------------- KOMBINASI SEARCH + SORT -----------------
def test_all_combinations():
    print("\n🔀 UJI KOMBINASI SORTING + SEARCHING\n")

    combinations = [
        ("Jump Search", jump_search, True, "Heap Sort", heap_sort),
        ("Jump Search", jump_search, True, "Merge Sort", merge_sort),
        ("Hash Search", hash_search, False, "Selection Sort", selection_sort),
        ("Jump Search", jump_search, True, "Bubble Sort", bubble_sort),
        ("Binary Search", binary_search, True, "Counting Sort", counting_sort),
    ]

    total_times = {}  # Untuk menyimpan total waktu dari setiap kombinasi

    for col, values in data_dict.items():
        if not values:
            continue
        print(f"📂 Kolom: {col}")
        target = values[len(values) // 2]  # Nilai tengah sebagai target

        for s_name, s_func, s_needs_sort, sort_name, sort_func in combinations:
            combo_key = f"{s_name} + {sort_name}"
            test_data = values.copy()

            # Counting Sort: periksa apakah bisa dilakukan
            if sort_name == "Counting Sort":
                try:
                    _ = list(map(int, test_data))
                    can_counting_sort = True
                except ValueError:
                    can_counting_sort = False
                if not can_counting_sort:
                    print(f"  ⚠️ {sort_name} tidak bisa untuk kolom ini (data bukan integer)")
                    # Tetap lakukan searching
                    search_start = time.time()
                    idx = s_func(test_data, target)
                    search_time = time.time() - search_start
                    total = search_time
                    total_times[combo_key] = total_times.get(combo_key, 0) + total
                    print(f"  🔁 {combo_key} → Index: {idx}, Sort: skipped, Search: {search_time:.6f}s")
                    continue

            # Sorting
            sort_start = time.time()
            sorted_data = sort_func(test_data)
            sort_time = time.time() - sort_start

            # Validasi hasil sorting jika diperlukan
            #if s_needs_sort and sorted_data != sorted(test_data):
             #   print(f"  ⚠️ Kombinasi {sort_name} + {s_name} gagal (data tidak terurut benar)")
              #  continue

            # Searching
            search_start = time.time()
            idx = s_func(sorted_data if s_needs_sort else test_data, target)
            search_time = time.time() - search_start

            # Simpan total waktu
            total = sort_time + search_time
            total_times[combo_key] = total_times.get(combo_key, 0) + total

            print(f"  🔁 {combo_key} → Index: {idx}, Sort: {sort_time:.6f}s, Search: {search_time:.6f}s")

        print()

    # Cetak total waktu kombinasi
    print("\n📊 TOTAL WAKTU SETIAP KOMBINASI:")
    for combo, total in total_times.items():
        print(f"  ⏱️ {combo}: {total:.6f} detik")


# ----------------- JALANKAN UJI -----------------
test_all_combinations()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🔀 UJI KOMBINASI SORTING + SEARCHING

📂 Kolom: product_id
  🔁 Jump Search + Heap Sort → Index: 14787, Sort: 0.192872s, Search: 0.000104s
  🔁 Jump Search + Merge Sort → Index: 14787, Sort: 0.193597s, Search: 0.000099s
  🔁 Hash Search + Selection Sort → Index: 16475, Sort: 44.884145s, Search: 0.006731s
  🔁 Jump Search + Bubble Sort → Index: 14787, Sort: 93.344146s, Search: 0.000077s
  ⚠️ Counting Sort tidak bisa untuk kolom ini (data bukan integer)
  🔁 Binary Search + Counting Sort → Index: 16475, Sort: skipped, Search: 0.000006s

📂 Kolom: product_category_name
  🔁 Jump Search + Heap Sort → Index: 3383, Sort: 0.136962s, Search: 0.000032s
  🔁 Jump Search + Merge Sort → Index: 3383, Sort: 0.159439s, Search: 0.000033s
  🔁 Hash Search + Selection Sort → Index: 32316, Sort: 40.070279s, Search: 0.002252s
  🔁 Jump Search + Bubble Sort → Index: 3383, Sort: 70.032777s, 